## NLPoetry

### Generating poems using word embeddings from specified poets
#### Inspired by [Benjamín Durán's AI poetry](https://towardsdatascience.com/creating-a-poems-generator-using-word-embeddings-bcc43248de4f)


In [170]:
import os
import pandas as pd
import numpy as np
import re
import spacy
import urllib.request
from bs4 import BeautifulSoup as bs
import pandas as pd
import random

writer_list = {
    "Maya Angelou" : "maya-angelou-poems",
    "Langston Hughes" : "langston-hughes-poems",
    "Pablo Neruda" : "pablo-neruda-poems",
    "William Wordsworth" : "william-wordsworth-poems",
    "William Shakespeare" : "william-shakespeare-poems",
    "Charles Bukowski" : "charles-bukowski-poems",
    "Robert Burns" : "robert-burns-poems",
    "Rabindranath Tagore" : "rabindranath-tagore-poems",
    "Rudyard Kipling" : "rudyard-kipling-poems",
    "Robert Frost" : "robert-frost-poems",
    "Edgar Allan Poe" : "edgar-allan-poe-poems",
    "Sylvia Plath" : "sylvia-plath-poems",
    "Walt Whitman" : "walt-whitman-poems",
    "William Blake" : "william-blake-poems",
    "Dylan Thomas" : "dylan-thomas-poems",
    "John Keats" : "john-keats-poems",
    "Wilfred Owen" : "wilfred-owen-poems",
    "Roald Dahl" : "roald-dahl-poems"
}

class AppURLopener(urllib.request.FancyURLopener): 
    version = "Mozilla/5.0" 
opener = AppURLopener()
writer = random.choice(list(writer_list.values()))
print(writer)
data = opener.open('https://mypoeticside.com/poets/' + writer).read().decode()

poem_list = bs(data, 'html.parser').find(class_="list-poems")
results = ["https:"+link.get('href') for link in poem_list.findAll('a')]

titles = []
content = []

for page in results:
     data = opener.open(page).read().decode()
     soup = bs(data, 'html.parser')
    
     poem_title = soup.find(class_='title-poem')
     poem_content = soup.find(class_='poem-entry')
    
     titles.append(poem_title.getText())
     print(poem_title.getText())
    
     content.append(poem_content.find('p').getText())
     
 #saves to a .csv file all the poems   
poems = pd.DataFrame({'title' : titles, 'text' : content})
csv_name = (writer + '.csv').replace("-", "_")
poems.to_csv(csv_name)

def poem_parsing(file, split = r"\n"):
    docs = pd.read_csv(os.getcwd() + "/" + file)
    num_docs = docs.shape[0]
    sentences = pd.DataFrame(columns=['doc_id', 'sentence'])
    
    for i in range(num_docs):
        poem = docs.text[i].lower()
        
        change_chars = {
            '?«' :  '«', 
            '(' :  '', 
            ')' : '', 
            ':' : ',', 
            '.' : ',', 
            ',,,' : ',', 
            '"' : ''
        }
        
        for x, y in change_chars.items():
            poem = poem.replace(x,y)
        
        s = re.split(split, poem)
        doc_id = [i] * len(s)
        sen_dic = {'doc_id' : doc_id, 'sentence' : s}
        
        sentences = sentences.append(pd.DataFrame(sen_dic))
    print(sentences)
        
    sentences = sentences[sentences.sentence != '']
    sentences.reset_index(drop=True, inplace=True)  
    sentences.to_csv("sentences_" + file)
    
poem_parsing(csv_name)


/Users/jordanramos/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods


john-keats-poems
A Dream, After Reading Dante's Episode...
Addressed To Haydon
Answer To A Sonnet By J.H.Reynolds
Bards of Passion and of Mirth,...
Bright Star
Endymion: Book I. A Thing of Beauty is a Joy Forever
Endymion: Book II
Endymion: Book III
Endymion: Book IV
Epistle To My Brother George
Fancy
Fill For Me A Brimming Bowl
Fragment of an Ode to Maia
Give Me Women, Wine, and Snuff
Happy Is England
His Last Sonnet
Hither, Hither, Love
How Many Bards Gild The Lapses Of Time!
Hymn To Apollo
Hyperion
I Stood tip-toe upon a little hill
If by Dull Rhymes our English must be Chain'd
In Drear-Nighted December
Isabella or The Pot of Basil
Keen, Fitful Gusts are Whisp'ring Here and There
La Belle Dame Sans Merci
Lamia
Lines
Lines on The Mermaid Tavern
Meg Merrilies
O Blush Not So!
O Solitude! If I Must With Thee Dwell
Ode On A Grecian Urn
Ode On Indolence
Ode on Melancholy
Ode To A Nightingale
Ode to Apollo
Ode To Autumn
Ode to Fanny
Ode To Psyche
On Fame
On First Looking Into Chapman's Hom

FileNotFoundError: [Errno 2] File b'/Users/jordanramos/Documents/NLPoetry/john_keats_poems.csv' does not exist: b'/Users/jordanramos/Documents/NLPoetry/john_keats_poems.csv'

In [169]:
def poem_generator(file, word, num_sen=12):
    spacy_nlp = spacy.load("en")
    subject = spacy_nlp(word)
    sentences = pd.read_csv(os.getcwd() +'/'+ file).fillna("")
    poem_id = int()
    poem_lst = []
    
    for i in range(num_sen):
        rand = np.random.randint(0, sentences.shape[0], size = 30)
        docs = spacy_nlp.pipe(list(sentences.sentence.iloc[rand]))
        
        similarities = []
        for s in docs:
            similarities.append(spacy_nlp(word).similarity(s))
            
        s_dict = {
            'similarity' : similarities,
            'doc_id' : sentences.doc_id.iloc[rand]
        }
        
        df_sim = pd.DataFrame(s_dict, index=rand)
        df_sim = df_sim[df_sim.doc_id != poem_id]
        df_sim.sort_values(by='similarity', inplace=True, ascending=False)
        
        s = sentences.sentence[df_sim.index[0]]
        
        change_chars = {
            '\n' :  '', 
            '\r' :  '', 
        }
        
        for x, y in change_chars.items():
            s = s.replace(x, y)
        
        poem_lst.append(s)
        poem_id = df_sim.doc_id.iloc[0]
        subject = spacy_nlp(s)
    
    poem = ("\n".join(poem_lst))
    return poem

In [165]:
def poem_format(text):
    return (text[:1].upper() + text[1:])[:-1] + '.'

In [167]:
sentences_csv = "sentences/sentences_" + csv_name

poem = poem_generator(file = sentences_csv, word='valor')
poem = poem_format(poem)
print(poem)

/Users/jordanramos/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  


Unspoken to
my plight is just as terrible,
disgust,
at their best, there is gentleness in humanity,
ashamed of my sentimentality and
telling you to forget the dead armies and the loves
whose life had
sadness takes me all over
and the sadness becomes so great
this is not a god-damned
from her! her precious wallace! christ! what a mess! he claimed he loved me,
letting the warmth o.
